# Overview 

**Dans ce notebook, on crée notre agent IA et on le prompt pour avoir une solution au porblème de Veolia**

In [53]:
import boto3
import os
import time
import json

Test pour créer une session et voir si ca marche correctement:

In [54]:
session = boto3.Session()
print(session)

Session(region_name='us-west-2')


On va maintenant lister tous les modèles LLM disponibles dans notre région:

In [55]:
bedrock = boto3.client('bedrock', region_name='us-west-2') 
models = bedrock.list_foundation_models()

for model in models['modelSummaries']:
    print(f"Model ID: {model['modelId']}")

Model ID: amazon.titan-tg1-large
Model ID: amazon.titan-embed-g1-text-02
Model ID: amazon.titan-text-lite-v1:0:4k
Model ID: amazon.titan-text-lite-v1
Model ID: amazon.titan-text-express-v1:0:8k
Model ID: amazon.titan-text-express-v1
Model ID: amazon.nova-pro-v1:0
Model ID: amazon.nova-lite-v1:0
Model ID: amazon.nova-micro-v1:0
Model ID: amazon.titan-embed-text-v1:2:8k
Model ID: amazon.titan-embed-text-v1
Model ID: amazon.titan-embed-text-v2:0
Model ID: amazon.titan-embed-image-v1:0
Model ID: amazon.titan-embed-image-v1
Model ID: amazon.titan-image-generator-v1:0
Model ID: amazon.titan-image-generator-v1
Model ID: amazon.titan-image-generator-v2:0
Model ID: amazon.rerank-v1:0
Model ID: stability.stable-diffusion-xl-v1:0
Model ID: stability.stable-diffusion-xl-v1
Model ID: stability.sd3-large-v1:0
Model ID: stability.sd3-5-large-v1:0
Model ID: stability.stable-image-core-v1:0
Model ID: stability.stable-image-core-v1:1
Model ID: stability.stable-image-ultra-v1:0
Model ID: stability.stable

On va à présent établir une connection à Amazon Redshift, notre base de données serverless:

In [56]:
# Initialisation du client Redshift Data
client = boto3.client('redshift-data', region_name='us-west-2')

# Paramètres de connexion
database = 'dev'
workgroup_name = 'redshift-wg-hackathon'

# Requête simple pour tester la connexion
sql_query = 'SELECT * from clients_database;'

try:
    response = client.execute_statement(
        Database=database,
        WorkgroupName=workgroup_name, 
        Sql=sql_query
    )
    print("Connexion réussie !")
except Exception as e:
    print(f"Erreur de connexion : {e}")

Connexion réussie !


On va test de faire une requête SQL à Amazon Redshift (notre base de données) et afficher le résultat de la requête: 

In [57]:
sql_query = 'SELECT * FROM clients_database LIMIT 10;' # on limite à 10 lignes

try:
    # Exécuter la requête SQL
    response = client.execute_statement(
        Database=database,
        WorkgroupName=workgroup_name, 
        Sql=sql_query
    )
    
    # Récupérer l'ID de l'exécution de la requête
    statement_id = response['Id']
    print(f"Requête envoyée, ID : {statement_id}")

    # Attendre que la requête soit terminée
    while True:
        status_response = client.describe_statement(Id=statement_id)
        status = status_response['Status']

        if status in ['FINISHED', 'FAILED', 'ABORTED']:
            break
        print("En attente des résultats...")
        time.sleep(2)  # Pause avant la prochaine vérification

    # Vérifier si la requête s'est bien exécutée
    if status == 'FINISHED':
        # Récupérer les résultats
        result_response = client.get_statement_result(Id=statement_id)
        records = result_response.get('Records', [])

        # Afficher les résultats
        if records:
            print("\nRésultats de la requête :")
            for row in records:
                print([col.get('stringValue', 'NULL') for col in row])  # Adaptation pour afficher chaque ligne
        else:
            print("Aucun résultat trouvé.")

    else:
        print(f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}")

except Exception as e:
    print(f"Erreur de connexion ou d'exécution : {e}")

Requête envoyée, ID : be93321d-9379-49b2-9f0e-ecb33ac97a60
En attente des résultats...
En attente des résultats...

Résultats de la requête :
['9901', '0', '1', '1', '1', '27257', '470', 'Good', '0', 'NULL', 'NULL', 'NULL', 'NULL', '4', '0', 'Unknown', 'High', 'Moved', '2025-03-09', '26', '18000', 'Unemployed', '1', 'Own', 'Single-family home', '1', 'Suburban', 'Public transportation', '2025-12-03', '226', '0']
['9902', '0', '0', '1', '0', '75763', '360', 'Poor', '3', 'Theft', '2024-04-06', 'Standard', '0.37', '1', '1', 'Fragile', 'Poor', 'Moved', '2024-12-27', '57', '18000', 'On-site', '1', 'Rent', 'Townhouse', '0', 'Suburban', 'Car', '2024-09-27', '257', '0']
['9903', '0', '0', '0', '1', '51306', '842', 'Good', '0', 'NULL', 'NULL', 'NULL', 'NULL', '6', '0', 'Unknown', 'High', 'Health Issue', '2024-03-07', '67', '18000', 'Remote', '1', 'Own', 'Townhouse', '1', 'Urban', 'Public transportation', '2024-10-30', '420', '0']
['9904', '1', '1', '0', '1', '55563', '260', 'Average', '0', 'NULL

## On va maintenant passer à la création de notre agent et aux prompts

Configurations: 

In [58]:
# ---- AWS CONFIGURATION ----
AWS_REGION = "us-west-2"
MODEL_ID = ""  # Utilisation de Mistral AI

# ---- REDSHIFT SERVERLESS CONFIGURATION ----
DATABASE = 'dev'  # nom de notre base de données
WORKGROUP_NAME = 'redshift-wg-hackathon'  # notre workgroup

# ---- INITIALISATION DES CLIENTS ----
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
redshift_client = boto3.client("redshift-data", region_name=AWS_REGION)

In [59]:
# ---- FONCTION POUR RÉCUPÉRER UN ÉCHANTILLON D'UNE TABLE ----
def get_table_sample():
    """Récupère 100 lignes d'une table Redshift pour analyse."""
    sql_query = f"SELECT * FROM clients_database WHERE Last_Account_Update = 1;"
    
    try:
        print(f"🔄 Envoi de la requête à Redshift: {sql_query}")
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        print(f"✅ Requête envoyée, ID: {statement_id}")

        # Timeout après 60 secondes
        start_time = time.time()
        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]

            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            
            # Vérification du timeout (60 secondes max)
            if time.time() - start_time > 60:
                print("⏳ Timeout dépassé (60s). Annulation de la requête.")
                return "Timeout: La requête a pris trop de temps."

            print("⏳ En attente des résultats...")
            time.sleep(3)  # Vérification toutes les 3 secondes

        if status == "FINISHED":
            print("✅ Requête terminée, récupération des résultats...")
            result_response = redshift_client.get_statement_result(Id=statement_id)
            records = [
                ", ".join([col.get('stringValue', 'NULL') for col in row])
                for row in result_response.get("Records", [])
            ]
            return records if records else "Aucune donnée trouvée."
        else:
            print(f"❌ Erreur d'exécution: {status}")
            return f"Erreur lors de l'exécution: {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        print(f"❌ Erreur de connexion ou d'exécution : {str(e)}")
        return f"Erreur : {str(e)}"
    

get_table_sample()

🔄 Envoi de la requête à Redshift: SELECT * FROM clients_database WHERE Last_Account_Update = 1;
✅ Requête envoyée, ID: 0d6f8053-6639-4c40-ab12-137495698742
⏳ En attente des résultats...
✅ Requête terminée, récupération des résultats...


['6253, 0, 1, 1, 1, 81978, 371, Excellent, 0, NULL, NULL, NULL, NULL, 1, 0, Good, Medium, Bought a Car, 2023-10-29, 52, 18000, Unemployed, 1, Own, Townhouse, 1, Urban, Biking, 2024-01-04, 336, 1',
 '4743, 1, 0, 0, 1, 86413, 427, Poor, 0, NULL, NULL, NULL, NULL, 2, 0, Fragile, Poor, Bought a Property, 2023-07-22, 43, 18000, Hybrid, 1, Own, Single-family home, 1, Urban, Biking, 2024-12-19, 167, 1',
 '4685, 1, 1, 0, 0, 46627, 758, Good, 0, NULL, NULL, NULL, NULL, 5, 0, Good, Poor, New Baby, 2024-04-04, 24, 35000, Remote, 1, Rent, Townhouse, 0, Urban, Car, 2024-06-06, 253, 1',
 '4522, 1, 1, 1, 0, 53395, 683, Good, 0, NULL, NULL, NULL, NULL, 18, 0, Medium, Poor, Moved, 2024-10-10, 43, 35000, Remote, 0, Own, Apartment, 0, Urban, Biking, 2023-07-16, 484, 1',
 '1732, 0, 1, 0, 0, 45841, 832, Average, 0, NULL, NULL, NULL, NULL, 6, 0, Fragile, High, Moved, 2023-11-11, 70, 35000, Unemployed, 1, Own, Apartment, 0, Urban, Walking, 2023-05-30, 249, 1']

In [60]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [61]:
pip install utils

Note: you may need to restart the kernel to use updated packages.


In [62]:
from PyPDF2 import PdfReader

def load_pdf_text(path):
    reader = PdfReader(path)
    text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    return text

In [63]:
def create_prompt(client_rows, column_doc, contracts_doc):
    header = (
        "You are an AI agent working for an insurance company.\n"
        "You have access to:\n"
        "- the client data from the 'clients_database' table\n"
        "- a document explaining each column ('Explanations about each columns of the clients dataset')\n"
        "- a document describing the available insurance contracts and their eligibility criteria ('The different types of insurance contracts and details')\n\n"
        "For each client below, identify the most suitable insurance contract based on their data and the criteria described. "
        "Return the results in the following format:\n\n"
        "- Client_ID: <ID>\n"
        "- Reasons: <Value> (<ColumnName>), ...\n"
        "- Suggested Contract: <Contract Name>\n\n"
    )

    client_data_section = "\n=== Client Data ===\n"
    for row in client_rows:
        client_data_section += f"{row}\n"

    docs_section = "\n=== Column Documentation ===\n" + column_doc
    contracts_section = "\n=== Contract Descriptions ===\n" + contracts_doc

    return header + client_data_section + docs_section + contracts_section

In [ ]:
def analyze_clients_with_claude():
    client_rows = get_table_sample()
    print(client_rows)
    if not isinstance(client_rows, list):
        print("❌ Erreur dans la récupération des clients.")
        return

    columns_text = load_pdf_text("Explanations_about_each_columns_of_the_clients_dataset.pdf")
    contracts_text = load_pdf_text("The_different_types_of_insurance_contracts_and_details.pdf")

    prompt = create_prompt(client_rows, columns_text, contracts_text)

    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 3000,
        "temperature": 0.3,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    try:
        print("📤 Envoi à Claude 3.5 Haiku via Bedrock...")
        response = bedrock.invoke_model(
            modelId="anthropic.claude-3-5-haiku-20241022-v1:0",
            body=json.dumps(request_body)
        )
        result = json.loads(response['body'].read())
        return result['content'][0]['text']

    except Exception as e:
        print(f"❌ Erreur avec Claude : {e}")
        return f"Erreur : {str(e)}"
    

analyze_clients_with_claude()


🔄 Envoi de la requête à Redshift: SELECT * FROM clients_database WHERE Last_Account_Update = 1;
✅ Requête envoyée, ID: 92233ad7-0c8c-4e0b-ad23-b1919ec61a02
⏳ En attente des résultats...
✅ Requête terminée, récupération des résultats...
['6253, 0, 1, 1, 1, 81978, 371, Excellent, 0, NULL, NULL, NULL, NULL, 1, 0, Good, Medium, Bought a Car, 2023-10-29, 52, 18000, Unemployed, 1, Own, Townhouse, 1, Urban, Biking, 2024-01-04, 336, 1', '4743, 1, 0, 0, 1, 86413, 427, Poor, 0, NULL, NULL, NULL, NULL, 2, 0, Fragile, Poor, Bought a Property, 2023-07-22, 43, 18000, Hybrid, 1, Own, Single-family home, 1, Urban, Biking, 2024-12-19, 167, 1', '4685, 1, 1, 0, 0, 46627, 758, Good, 0, NULL, NULL, NULL, NULL, 5, 0, Good, Poor, New Baby, 2024-04-04, 24, 35000, Remote, 1, Rent, Townhouse, 0, Urban, Car, 2024-06-06, 253, 1', '4522, 1, 1, 1, 0, 53395, 683, Good, 0, NULL, NULL, NULL, NULL, 18, 0, Medium, Poor, Moved, 2024-10-10, 43, 35000, Remote, 0, Own, Apartment, 0, Urban, Biking, 2023-07-16, 484, 1', '1732

"I'll analyze each client and recommend the most suitable insurance contract based on their data:\n\n1. Client ID: 6253\n- Reasons: Bought a Car (Recent Life Event), Urban Location, Has Car (1)\n- Suggested Contract: Comprehensive Auto Coverage\nRationale: Recent car purchase, urban environment, and car ownership suggest need for robust auto insurance with comprehensive protection.\n\n2. Client ID: 4743\n- Reasons: Bought a Property (Recent Life Event), Poor Creditworthiness, Fragile Health, Urban Location\n- Suggested Contract: Renter's Insurance\nRationale: Recently bought a property, fragile health indicates need for protection, and current living status suggests renter's insurance is most appropriate.\n\n3. Client ID: 4685\n- Reasons: New Baby (Recent Life Event), Remote Work, Medium Income, Urban Location\n- Suggested Contract: Family Health Plan\nRationale: New baby, need for comprehensive health coverage, family-oriented life stage suggests family health insurance.\n\n4. Client 

In [65]:
result = analyze_clients_with_claude()
print(result)

🔄 Envoi de la requête à Redshift: SELECT * FROM clients_database WHERE Last_Account_Update = 1;
✅ Requête envoyée, ID: 5c294dad-cafb-4c83-974e-6fc980c6918b
⏳ En attente des résultats...
✅ Requête terminée, récupération des résultats...
['6253, 0, 1, 1, 1, 81978, 371, Excellent, 0, NULL, NULL, NULL, NULL, 1, 0, Good, Medium, Bought a Car, 2023-10-29, 52, 18000, Unemployed, 1, Own, Townhouse, 1, Urban, Biking, 2024-01-04, 336, 1', '4743, 1, 0, 0, 1, 86413, 427, Poor, 0, NULL, NULL, NULL, NULL, 2, 0, Fragile, Poor, Bought a Property, 2023-07-22, 43, 18000, Hybrid, 1, Own, Single-family home, 1, Urban, Biking, 2024-12-19, 167, 1', '4685, 1, 1, 0, 0, 46627, 758, Good, 0, NULL, NULL, NULL, NULL, 5, 0, Good, Poor, New Baby, 2024-04-04, 24, 35000, Remote, 1, Rent, Townhouse, 0, Urban, Car, 2024-06-06, 253, 1', '4522, 1, 1, 1, 0, 53395, 683, Good, 0, NULL, NULL, NULL, NULL, 18, 0, Medium, Poor, Moved, 2024-10-10, 43, 35000, Remote, 0, Own, Apartment, 0, Urban, Biking, 2023-07-16, 484, 1', '1732

# ----------------------------------------------------------

In [66]:
# ---- FONCTION POUR DÉTECTER LES ANOMALIES ----
def agent_detect_anomalies():
    """Détecte les anomalies d'une table spécifique dans Redshift via Mistral AI."""
    table_sample = get_table_sample()
    
    if isinstance(table_sample, str):
        return table_sample  # Si erreur, on la renvoie directement

    formatted_sample = "\n".join(table_sample)

    # Construction du prompt
    prompt = f"""
    Tu es un expert en qualité des données.
    {rep_prompt1}
    
    Voici un échantillon de la table "{table_name}":
    {formatted_sample}
    
    Identifie les anomalies (valeurs nulles, doublons, erreurs de format, etc.).
    Génère une requête SQL pour afficher les lignes contenant des données incorrectes.

    Réponds en suivant ce format :
    - **Type d'anomalie** : [Catégorie de l'anomalie]
    - **Description** : [Brève explication]
    - **Requête SQL** : [Requête pour afficher les données erronées]
    """

    return analyze_with_mistral(prompt)


# ---- FONCTION POUR INTERAGIR AVEC MISTRAL AI ----
def analyze_with_mistral(prompt):
    """Envoie un prompt à Mistral AI via Amazon Bedrock."""
    request_body = {
        "prompt": prompt,
        "max_tokens": 3000,
        "temperature": 0.3,
        "top_p": 0.9
    }

    try:
        print("🔄 Envoi du prompt à Mistral AI...")
        response = bedrock.invoke_model(
            modelId=MODEL_ID,
            body=json.dumps(request_body)
        )
        print("✅ Réponse reçue de Mistral.")

        response_body = json.loads(response["body"].read())
        return response_body.get("outputs", [{}])[0].get("text", "")

    except Exception as e:
        print(f"❌ Erreur d'interaction avec Mistral AI : {str(e)}")
        return f"Erreur : {str(e)}"


# ---- EXÉCUTION ----
if __name__ == "__main__":
    table_name = input("🔍 Entrez le nom de la table à analyser : ")
    response = agent_detect_anomalies()

    print("\n Rapport d'Anomalies \n")
    print(response)
    rep_prompt2 = response

🔄 Envoi de la requête à Redshift: SELECT * FROM clients_database WHERE Last_Account_Update = 1;
✅ Requête envoyée, ID: 47b73e62-a645-4b5f-999a-5598828d6494
⏳ En attente des résultats...
✅ Requête terminée, récupération des résultats...
🔄 Envoi du prompt à Mistral AI...
❌ Erreur d'interaction avec Mistral AI : Parameter validation failed:
Invalid length for parameter modelId, value: 0, valid min length: 1

 Rapport d'Anomalies 

Erreur : Parameter validation failed:
Invalid length for parameter modelId, value: 0, valid min length: 1
